##### pysam docs

https://pysam.readthedocs.io/en/latest/index.html

In [1]:
import numpy as np
import pysam
from collections import Counter
import csv

## Need to set some variables here:

    -- SAM files
    -- freqs
    -- ref names
    -- % AR to simulate

#### Set it such that A is head and B is tails (** will need to modify later **)

In [2]:
samfile_A = pysam.AlignmentFile("pol_a.sam", "r")
all_reads_A = list(samfile_A.fetch())

samfile_B = pysam.AlignmentFile("pol_b.sam", "r")
all_reads_B = list(samfile_B.fetch())

In [3]:
print("reads in A --> ", len(all_reads_A))
print("reads in B --> ", len(all_reads_B))

reads in A -->  10000
reads in B -->  10000


#### determining the number of times to loop over to generate a read 

In [4]:
per_15 = int(len(all_reads_A) * .15)
over_estimate = int(len(all_reads_A) * .50)

other_reads_a = int(len(all_reads_A) - per_15)
other_reads_b = int(len(all_reads_B) - per_15)


#### Gathering the random reads we want mess with

In [5]:
sampled_read_data = []
for number in range(over_estimate):
    #print(number)
    seq_grabber = np.random.randint(len(all_reads_A))
    #print(seq_grabber)
    temp = {}
    temp[all_reads_A[seq_grabber], number] = seq_grabber
    sampled_read_data.append(temp)
  
print(list(sampled_read_data[0].values())[0])
#B.CH.2002.HIV_CH_BID-V3527_2002.JQ403021-774
print(len(sampled_read_data))
print(list(sampled_read_data[0].keys())[0][0].qual)

3600
5000
GGGGGGGDGGGGGGGGGGCGGG0CFGGGGGGG/>GGGGGGFGGGGBGEG=GG1GGGGGGGGGGGGGGCFGGGGGGCGGGGDG1GG1GGGCGGGGGGGGG0GGGGGGGGDDGG1GGC0BCB


### This is just to check about how many matching reads we are finding

In [6]:

count_found = 0
good_reads = []
for i in sampled_read_data:
    read_a = list(i.keys())[0][0]
    ind = list(i.values())[0]

    original_ind = (read_a.reference_start, read_a.reference_end)
    #print(original_ind)
    
    read_b = all_reads_B[ind]
    read_b_ind = (read_b.reference_start, read_b.reference_end)
    #print(read_b_ind)
    
    if original_ind == read_b_ind:
        good_reads.append((read_a, read_b))
        count_found += 1
    else:
        continue
    #read_b = 
    
    if count_found == per_15:
        #print(count_found)
        print("we have all the seqs we need!!")
        break
        
#print(count_found)
print(len(good_reads))
#good_reads

we have all the seqs we need!!
1500


#### dont like that we are passing around this large list 

In [7]:
'''
                                            0                                        1 
                                    0                                           1    
tuple 0 = ((<pysam.libcalignedsegment.AlignedSegment object at 0x7f4f6fd39a68>, 0), 8535)

'''
# count = 0
# count_a_b = 0
with open(".fastq", "w") as out:
    for i in range(len(all_reads_A)):
        if i < len(good_reads):
            # write AR reads to file
            #count += 1
        
            ## randomly grab a seq ## 
            seq_getter = np.random.randint(len(good_reads))
            temp = good_reads[seq_getter]
            r_a, r_b = temp[0], temp[1]

            #print(count, ": match: comparing " + r_a.qname + " to " + r_b.qname + "\n")
            '''
            here I need to:
                1. index one seq from the other
                2. combine the seqs
                    a. label
                    b. joined seq
                    c. joined q-score

            '''
            seq_splitter = np.random.randint(len(r_a.seq))
            print(seq_splitter)
            r_start = r_a.seq[:seq_splitter]
            r_end = r_b.seq[seq_splitter:]

            joined_label = r_a.qname + r_b.qname
            joined_seq = r_start+r_end
            joined_q = r_a.qual[:seq_splitter] + r_b.qual[seq_splitter:]

            
            #label = "some_label"
            #joined_seq = "seq-ACTG"
            #joined_q = "q-asdflk"

            out.write("{}\n{}\n+\n{}\n".format(joined_label, joined_seq, joined_q)) 
            
        
#         if count == 10:
#             print("all done")
#             break
            
        else:
            ## jsut write everything else here
            seq_getter = np.random.randint(len(all_reads_A))
            r_a = all_reads_A[seq_getter]
            r_b = all_reads_B[seq_getter]
            
            out.write("{}\n{}\n+\n{}\n".format(read_a.qname, r_a.seq, r_a.qual)) 
            out.write("{}\n{}\n+\n{}\n".format(read_b.qname, r_b.seq, r_b.qual)) 

        


NameError: name 'count' is not defined

In [ ]:
#master = {"pol_a": (.5, all_reads_A), "pol_b":(.5, all_reads_B)}

In [ ]:
'''
f = list(master.keys())
freqs = np.array(f)

## choice from freq ##
i = np.random.choice(2, 1, True, freqs)[0]
original_ref = f[i]

## re order the list based on the freqs
freq_reweight_list = freqs[np.arange(len(freqs)) !=i]
freq_reweight = freq_reweight_list / np.sum(freq_reweight_list)
j = np.random.choice(len(freq_reweight), 1, True, freq_reweight)[0]

## this will return reads that match        
good_reads = list(filter(
lambda read: read.reference_start == read_a.reference_start and read.reference_end == read_a.reference_end,
reads_b
))

## this might be useful to index each sequence against each other 
grabber = np.random.randint(len(seq_red_2))
print("random grabber --> ", grabber)

indexed_seq_1_start = seq_read_1[:grabber]
indexed_seq_2_end = seq_read_2[grabber:]
'''
